SAM 2024

#TP10 LSM Store

date document: 25/04/2024

## BOUZOURINE Hichem

L'objectif est de comprendre, par la pratique, les principes du stockage couramment mis en oeuvre dans les systemes big data tels que Cassandra et HBase.

## Préparation

In [1]:
import os
import shutil
import random
import numpy as np
import time
from sortedcontainers import SortedDict

os.makedirs("data", exist_ok=True)

In [3]:
# !rm -f *.csv

zsh:1: no matches found: *.csv


# Classe LSM : Log Structured Merge Storage

La classe LSM définit les principales fonctions du stockage basé sur LSM.

In [4]:
class LSM:
  D_MAXSIZE =10


  def __init__(self):
    self.D = SortedDict()
    self.first_image = 0
    self.last_image = 0
    self.MAX_IMAGE = 2

  def set_max_image(self, m):
    self.MAX_IMAGE= m


  def update(self, k, v):
      self.D[k] = v

      # vider D dans un fichier si D a atteint sa taille maxi
      if len(self.D) == self.D_MAXSIZE:
        self.flush_D()
        # continuer avec un nouveau D vide
        self.D = SortedDict()


  def select(self, k):
    if k in self.D:
      return self.D[k]

    # recherche parmi les images du niveau L0
    nb_image = self.last_image - self.first_image + 1
    for m in range(nb_image):
      with open(f"data/image{self.last_image - m}.csv", "r") as f:
        for line in f:
          line = line.split(',')
          k1,v1 = int(line[0]), int(line[1])
          if k1 == k:
              return v1

    # recherche dans L1
    with open(f"data/L1.csv", "r") as f:
      for line in f:
        line = line.split(',')
        k1, v1 = int(line[0]), int(line[1])
        if k1 == k:
            return v1
    # not found
    return "NULL"



  def flush_D(self):
    if self.first_image == 0:
      self.first_image = 1

    self.last_image +=1

    with open(f"data/image{self.last_image}.csv", "w") as f:
      for k, v in self.D.items():
          f.write(str(k) + "," + str(v) + '\n')
      print(f"saved {len(self.D)} items in image {self.last_image}")


  def merge_L0_to_L1(self):
    # deplace les données de la premiere image de L0 vers le niveau L1
    nb_image = self.last_image - self.first_image + 1

    if nb_image > self.MAX_IMAGE:
      if os.path.isfile("data/L1.csv") :
        L1_new = []
        L0 = iter(self.read_log(f'image{self.first_image}.csv'))
        k0, line0 = next(L0, (None, ""))

        #fusionner L1 et l'image
        for k1, line1 in self.read_log("L1.csv"):
          while k0 is not None and k0<=k1:
              L1_new.append(line0)
              k0, line0 = next(L0, (None, ""))
          L1_new.append(line1)
        print('nb of lines in L1:', len(L1_new))

        # créer un nouveau fichier L1 contenant le resultat de la fusion
        with open("data/L1_new.csv", 'w') as file_L1_new:
          file_L1_new.write(''.join(L1_new))
        shutil.move("data/L1_new.csv", "data/L1.csv")
      else:
        # L1 ne contient rien, il suffit de déplacer l'image vers L1
        print(f"move image{self.first_image}.csv to L1.csv")
        shutil.move(f'data/image{self.first_image}.csv', "data/L1.csv")
      self.first_image +=1


  def read_log(self, file):
    with open("data/" + file) as f:
      for line in f:
        key = line.split(',')[0]
        yield int(key), line

print("class LSM defined")

class LSM defined


## Exercice 1

Comprendre la classe LSM définie ci dessus

## Ecriture de données dans le LSM



On effectue un test d'écriture de plusieurs couples dans le LSM

In [5]:
# on nettoie le dossier ou seront stockés les données
shutil.rmtree("data")
os.makedirs("data", exist_ok=True)


nb_update = 100

rng = np.random.default_rng(seed=1)
keys = rng.integers(0, 100, nb_update)
values = rng.integers(0, 1000, nb_update)

store = LSM()

store.set_max_image(2)   # remplacer 2 par 4 ?

t1 = time.time()
for k,v in zip(keys,values):
  store.update(k,v)
  store.merge_L0_to_L1() # essayer d'enlever cette ligne

print("duree", round( (time.time() - t1) * 1000, 0), "ms")


saved 10 items in image 1
saved 10 items in image 2
saved 10 items in image 3
move image1.csv to L1.csv
saved 10 items in image 4
nb of lines in L1: 20
saved 10 items in image 5
nb of lines in L1: 30
saved 10 items in image 6
nb of lines in L1: 39
saved 10 items in image 7
nb of lines in L1: 47
saved 10 items in image 8
nb of lines in L1: 56
saved 10 items in image 9
nb of lines in L1: 66
duree 9.0 ms


1) Que se passe-t-il si on ajoute
store.set_max_image(4) au début du test d'écriture ?

permettre jusqu'à 4 images dans le niveau L0 avant qu'elles ne soient fusionnées dans le niveau L1.

2) Que se pass-t-il si on enlève la ligne
store.merge_L0_to_L1() dans le test?

les données ne seront jamais fusionnées du niveau L0 vers le niveau L1. Cela signifie que toutes les données seront conservées dans le niveau L0 jusqu'à ce que le programme se termine

## Lecture

In [6]:
# Pour vérifier que la lecture est correcte, on connait à l'avance la valeur la plus récente de chaque k qu'on est censé lire
test_D = {}
for k,v in zip(keys,values):
  test_D[k]=v


t1 = time.time()
for k,v in test_D.items():
    # print("select", k)
    v1 = store.select(k)
    if v1 != v:
      print(f"Erreur pour la clé {k} ! La valeur retrouvée ({v1}) n'est pas celle attendue ({v})")

print("duree", round( (time.time() - t1) * 1000, 0), "ms")


Erreur pour la clé 97 ! La valeur retrouvée (NULL) n'est pas celle attendue (470)
Erreur pour la clé 96 ! La valeur retrouvée (NULL) n'est pas celle attendue (885)
duree 5.0 ms


# Exercice 2 : Effet de la fréquence de lecture

Proposer un test de lecture qui mette en évidence le fait que les clés lues fréquemment (par exemple une lecture toutes les 5 lectures) sont accédées plus rapidement que les clés rares (une lecture toutes les 50 lectures).

In [8]:

# Configuration des lectures
total_reads = 1000
freq_frequent_reads = 5  # Lecture fréquente
freq_rare_reads = 50     # Lecture rare

# Lecture fréquente
keys_frequent = np.random.choice(keys, total_reads // freq_frequent_reads)

# Lecture rare
keys_rare = np.random.choice(keys, total_reads // freq_rare_reads)

# Test de lecture
t1 = time.time()

print("Lecture fréquente:", len(keys_frequent))
# Lecture fréquente
for k in keys_frequent:
    v = test_D[k]
    v1 = store.select(k)
    if v1 != v:
        print(f"Erreur pour la clé {k} ! La valeur retrouvée ({v1}) n'est pas celle attendue ({v})")

print("Lecture rare:", len(keys_rare))
# Lecture rare
for k in keys_rare:
    v = test_D[k]
    v1 = store.select(k)
    if v1 != v:
        print(f"Erreur pour la clé {k} ! La valeur retrouvée ({v1}) n'est pas celle attendue ({v})")

print("Durée totale de lecture:", round((time.time() - t1) * 1000, 0), "ms")

Lecture fréquente: 200
Erreur pour la clé 96 ! La valeur retrouvée (NULL) n'est pas celle attendue (885)
Erreur pour la clé 96 ! La valeur retrouvée (NULL) n'est pas celle attendue (885)
Erreur pour la clé 96 ! La valeur retrouvée (NULL) n'est pas celle attendue (885)
Erreur pour la clé 97 ! La valeur retrouvée (NULL) n'est pas celle attendue (470)
Erreur pour la clé 97 ! La valeur retrouvée (NULL) n'est pas celle attendue (470)
Lecture rare: 20
Durée totale de lecture: 12.0 ms


# Exercice 3 : Amélioration du stockage du niveau L1

On voit que le niveau L1 contient un seul fichier `L1.csv` qui peut grandir indéfiniment.
Cela tend à ralentir la recherche d'une clé "peu fréquente" qui serait dans le niveau L1.

Proposer une classe LSM_v2 pour laquelle le niveau L1 est composé de plusieurs fichiers F1, F2, ... tels que
*   chaque fichier contient au maximium N lignes (N= nombre de lignes dans une image),
*   les clés sont globalement triées dans les fichiers : toutes les clés du fichier Fi sont inférieures à celles du fichiers Fj (avec i<j)



In [9]:
import os
import shutil
import numpy as np
import time
from sortedcontainers import SortedDict

class LSM_v2:
    D_MAXSIZE = 10
    MAX_IMAGE = 2
    MAX_LINES_PER_FILE = 10

    def __init__(self):
        self.D = SortedDict()
        self.first_image = 0
        self.last_image = 0
        self.L1_files = []

    def set_max_image(self, m):
        self.MAX_IMAGE = m

    def update(self, k, v):
        self.D[k] = v

        if len(self.D) == self.D_MAXSIZE:
            self.flush_D()
            self.D = SortedDict()

    def select(self, k):
        if k in self.D:
            return self.D[k]

        # Recherche dans les fichiers L1
        for file in self.L1_files:
            with open(file, "r") as f:
                for line in f:
                    k1, v1 = map(int, line.strip().split(','))
                    if k1 == k:
                        return v1

        return "NULL"

    def flush_D(self):
        if self.first_image == 0:
            self.first_image = 1

        self.last_image += 1
        image_filename = f"data/image{self.last_image}.csv"
        with open(image_filename, "w") as f:
            for k, v in self.D.items():
                f.write(f"{k},{v}\n")
        print(f"Saved {len(self.D)} items in image {self.last_image}")
        self.merge_L0_to_L1(image_filename)

    def merge_L0_to_L1(self, image_filename):
        self.L1_files.append(image_filename)
        if len(self.L1_files) > self.MAX_IMAGE:
            self.merge_files_L1()

    def merge_files_L1(self):
        merged_file = []
        for file in self.L1_files:
            merged_file += self.read_log(file)
            os.remove(file)
        self.L1_files = []

        # Trier les données fusionnées
        merged_file.sort(key=lambda x: int(x.split(',')[0]))

        # Créer des fichiers L1 avec un maximum de lignes par fichier
        num_files = len(merged_file) // self.MAX_LINES_PER_FILE
        if len(merged_file) % self.MAX_LINES_PER_FILE != 0:
            num_files += 1

        for i in range(num_files):
            start = i * self.MAX_LINES_PER_FILE
            end = min((i + 1) * self.MAX_LINES_PER_FILE, len(merged_file))  # Correction ici
            filename = f"data/L1_{i+1}.csv"
            with open(filename, 'w') as f:
                f.writelines(merged_file[start:end])
            self.L1_files.append(filename)  # Ajout des nouveaux fichiers à la liste


    def read_log(self, file):
        with open(file) as f:
            return f.readlines()

print("class LSM_v2 defined")

# Test d'utilisation de la classe LSM_v2
shutil.rmtree("data", ignore_errors=True)
os.makedirs("data", exist_ok=True)

nb_update = 100
rng = np.random.default_rng(seed=1)
keys = rng.integers(0, 100, nb_update)
values = rng.integers(0, 1000, nb_update)

store = LSM_v2()
store.set_max_image(2)

t1 = time.time()
for k, v in zip(keys, values):
    store.update(k, v)

print("Durée d'écriture:", round((time.time() - t1) * 1000, 0), "ms")

# Test de lecture
test_D = {k: v for k, v in zip(keys, values)}

t1 = time.time()
for k, v in test_D.items():
    v1 = store.select(k)
    if v1 != v:
        print(f"Erreur pour la clé {k} ! La valeur retrouvée ({v1}) n'est pas celle attendue ({v})")

print("Durée de lecture:", round((time.time() - t1) * 1000, 0), "ms")


class LSM_v2 defined
Saved 10 items in image 1
Saved 10 items in image 2
Saved 10 items in image 3
Saved 10 items in image 4
Saved 10 items in image 5
Saved 10 items in image 6
Saved 10 items in image 7
Saved 10 items in image 8
Saved 10 items in image 9
Durée d'écriture: 9.0 ms
Erreur pour la clé 3 ! La valeur retrouvée (873) n'est pas celle attendue (825)
Erreur pour la clé 82 ! La valeur retrouvée (555) n'est pas celle attendue (855)
Erreur pour la clé 86 ! La valeur retrouvée (479) n'est pas celle attendue (617)
Erreur pour la clé 42 ! La valeur retrouvée (81) n'est pas celle attendue (771)
Erreur pour la clé 27 ! La valeur retrouvée (219) n'est pas celle attendue (919)
Erreur pour la clé 40 ! La valeur retrouvée (861) n'est pas celle attendue (639)
Erreur pour la clé 64 ! La valeur retrouvée (840) n'est pas celle attendue (691)
Erreur pour la clé 83 ! La valeur retrouvée (919) n'est pas celle attendue (262)
Erreur pour la clé 32 ! La valeur retrouvée (645) n'est pas celle attendue

# Exercice 4 : fusion entre plusieurs images

On voit que les imagess de L0 sont fusionnées individuellement vers le niveau L1.
Proposer une solution pour fusionner k images de L0 vers L1.
Indication: tenir compte du fait que les images peuvent contenir des clés identiques.

In [10]:
import os
import shutil
import numpy as np
import time
from sortedcontainers import SortedDict

class LSM_v2:
    D_MAXSIZE = 10
    MAX_IMAGE = 2
    MAX_LINES_PER_FILE = 10

    def __init__(self):
        self.D = SortedDict()
        self.first_image = 0
        self.last_image = 0
        self.L1_files = []

    def set_max_image(self, m):
        self.MAX_IMAGE = m

    def update(self, k, v):
        self.D[k] = v

        if len(self.D) == self.D_MAXSIZE:
            self.flush_D()
            self.D = SortedDict()

    def select(self, k):
        if k in self.D:
            return self.D[k]

        # Recherche dans les fichiers L1
        for file in self.L1_files:
            with open(file, "r") as f:
                for line in f:
                    k1, v1 = map(int, line.strip().split(','))
                    if k1 == k:
                        return v1

        return "NULL"

    def flush_D(self):
        if self.first_image == 0:
            self.first_image = 1

        self.last_image += 1
        image_filename = f"data/image{self.last_image}.csv"
        with open(image_filename, "w") as f:
            for k, v in self.D.items():
                f.write(f"{k},{v}\n")
        print(f"Saved {len(self.D)} items in image {self.last_image}")
        self.merge_L0_to_L1(image_filename)

    def merge_L0_to_L1(self, image_filename):
        self.L1_files.append(image_filename)
        if len(self.L1_files) > self.MAX_IMAGE:
            self.merge_files_L1()

    def merge_files_L1(self):
        merged_file = self.collect_data_from_L0()
        if len(merged_file) > 0:
            # Trier les données fusionnées
            merged_file.sort(key=lambda x: int(x.split(',')[0]))

            # Créer des fichiers L1 avec un maximum de lignes par fichier
            num_files = len(merged_file) // self.MAX_LINES_PER_FILE
            if len(merged_file) % self.MAX_LINES_PER_FILE != 0:
                num_files += 1

            for i in range(num_files):
                start = i * self.MAX_LINES_PER_FILE
                end = min((i + 1) * self.MAX_LINES_PER_FILE, len(merged_file))
                filename = f"data/L1_{i+1}.csv"
                with open(filename, 'w') as f:
                    f.writelines(merged_file[start:end])
                self.L1_files.append(filename)

    def collect_data_from_L0(self):
        merged_data = {}
        for file in self.L1_files:
            data = self.read_log(file)
            for line in data:
                key, value = map(int, line.strip().split(','))
                merged_data[key] = value
            os.remove(file)
        self.L1_files = []
        return [f"{k},{v}\n" for k, v in merged_data.items()]

    def read_log(self, file):
        with open(file) as f:
            return f.readlines()

print("class LSM_v2 defined")

# Test d'utilisation de la classe LSM_v2
shutil.rmtree("data", ignore_errors=True)
os.makedirs("data", exist_ok=True)

nb_update = 100
rng = np.random.default_rng(seed=1)
keys = rng.integers(0, 100, nb_update)
values = rng.integers(0, 1000, nb_update)

store = LSM_v2()
store.set_max_image(2)

t1 = time.time()
for k, v in zip(keys, values):
    store.update(k, v)

print("Durée d'écriture:", round((time.time() - t1) * 1000, 0), "ms")

# Test de lecture
test_D = {k: v for k, v in zip(keys, values)}

t1 = time.time()
for k, v in test_D.items():
    v1 = store.select(k)
    if v1 != v:
        print(f"Erreur pour la clé {k} ! La valeur retrouvée ({v1}) n'est pas celle attendue ({v})")

print("Durée de lecture:", round((time.time() - t1) * 1000, 0), "ms")


class LSM_v2 defined
Saved 10 items in image 1
Saved 10 items in image 2
Saved 10 items in image 3
Saved 10 items in image 4
Saved 10 items in image 5
Saved 10 items in image 6
Saved 10 items in image 7
Saved 10 items in image 8
Saved 10 items in image 9
Durée d'écriture: 6.0 ms
Durée de lecture: 5.0 ms
